In [1]:
fp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/static analysis/hw-memory/benchmarks/big_spec/big_spec.asm"

seg_fp = r'/Users/gugujixiao/workspace/project/HWMemory/Code/static analysis/hw-memory/benchmarks/big_spec/big_specD.asm'

code_fp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/static analysis/hw-memory/benchmarks/big_spec/big_spec_l.asm"


In [2]:
'''读指令, 指令对象化'''
from newCFG.cfg import proc_identify
from newCFG.isa import Instruction, AddrMode
from typing import Tuple
from newCFG.read_asm import AsmFileReader, StatementType
from cache_analysis.read_segment import segmentReader
from newCFG.read_codeMix import AsmCodeReader, MixStatementType

reader =AsmFileReader(fp)
segreader = segmentReader(seg_fp)
#codereader = AsmCodeReader(code_fp)

statements = list()
code_stat = list()

for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)

"""for i in range(len(codereader.statements)):
    s: Tuple[MixStatementType, tuple]
    if codereader.statements[i][0] == MixStatementType.codeline:
        codeline = codereader.statements[i][1]
        addr = ""
        not_find = True
        k = i
        while not_find:
            k += 1
            if codereader.statements[k][0] == MixStatementType.asm:
                addr =  int(codereader.statements[k][1][0],16) 
                not_find = False

        code_stat.append((addr,codeline,"loopname"))"""
    

#for i in code_stat:
#    print(i)




procs = proc_identify(statements)
for proc in procs:
    print(proc.name, proc.beg_addr.hex_str())
    if False and proc.name == 'main':
        for inst in proc.instruction:
            print(inst.addr.hex_str(), inst.opcode, inst.name, inst.operands, inst.branch_info)
print()

_init 0000000000400508
.plt 0000000000400520
__libc_start_main@plt 0000000000400540
random@plt 0000000000400550
__gmon_start__@plt 0000000000400560
abort@plt 0000000000400570
putchar@plt 0000000000400580
_start 00000000004005c0
__wrap_main 00000000004005f4
_dl_relocate_static_pie 0000000000400600
call_weak_fn 0000000000400604
deregister_tm_clones 0000000000400620
register_tm_clones 0000000000400650
__do_global_dtors_aux 0000000000400690
frame_dummy 00000000004006c0
main 00000000004006c4
InitializeMesh_1 00000000004007c4
InitializeMesh_2 0000000000400958
CompareMesh 0000000000400aec
Mesh2 0000000000400bd8
SwapMesh 0000000000400d70
Mesh_dot 0000000000400e88
ComplexDataHandle 0000000000400fc0
ManiplateData1 0000000000400fe4
ManiplateData2 0000000000401114
_fini 0000000000401244



In [3]:
'''分proc'''
from graphviz import Digraph
from newCFG.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
g.render(filename='procedures', directory='./output', format='svg')

'output/procedures.svg'

In [4]:
'''处理call_graph'''
from newCFG.cfg import CallGraph

call_graph = CallGraph(procs)

g = call_graph.draw_graph()
g.render(filename='call_graph', directory='./output', format='svg')

'output/call_graph.svg'

In [5]:
'''生成TCFG和loop信息'''
from newCFG.cfg import TCfg


tcfg = TCfg(call_graph)
tcfg.build_tcfg()

tcfg_nodes = tcfg.all_nodes
tcfg_edges = tcfg.edges

g = tcfg.draw_graph()
g.render(filename='tcfg', directory='./output', format='svg')

#for node in tcfg_nodes:
#    node.set_rw_condition()
#    #print(node.name, node.base_proc.name, node.inst_range)
#    print(node.name, node.base_proc.name)
#    print("出边")
#    for i in node.outgoing_edge:
#        print(i.dst.name)
#    print("入边")
#    for i in node.incoming_edge:
#        print(i.src.name)
#
#    print()
tcfg.build_loop_hrchy()
# tcfg.add_loop_bound(r'D:\workspace\hw-memory\benchmarks\loop_bound.txt')
# tcfg.add_loop_bound(r'D:\workspace\Gitdocuments\hw-memory\benchmarks\loop_bound.txt')

"""loop_range =list()
with open("./output/cfg_addr.txt","w") as f:
    for l in tcfg.loops:
        #f.write(l.name+"\n")
        #print(l.name)
        for n in l.all_nodes:
            ins = n.instructions
            #head = ins[0].addr
            head,end = n.inst_range
            string = l.name+" "+head.hex_str()+" "+end.hex_str()+"\n"
            loop_range.append(([head.val(),end.val()],l.name))
            f.write(string)
        f.write("\n")


loop_code_range =list()


for i in code_stat:
    for l in loop_range:
        if l[0][0]<i[0]<l[0][1]:
            loop_code_range.append((i[0],i[1],l[1]))"""

#for i in loop_code_range:
#    print(i)


for l in tcfg.loops:
    print("{} (in {} children {}): {}->{} {}".format(l.name, l.father.name if l.father is not None else "None", [sub_l.name for sub_l in l.children], l.back_edge.src.name, l.back_edge.dst.name, [n.name for n in l.all_nodes]))
    l.back_edge.is_backEdge = True
for n in tcfg.all_nodes:
     temp = n.inside_loop.name if n.inside_loop is not None else 'None'
     print("{} {}".format(n.name, temp))



#for n in tcfg_nodes:
#    print(n.name,"n的值",n.node_value)
        

#for e in tcfg_edges:
#    print(e.src.name,e.dst.name,e.is_backEdge)




l2 (in l1 children []): n104->n105 ['n104', 'n105']
l1 (in l0 children []): n106->n107 ['n103', 'n104', 'n105', 'n106', 'n107']
l6 (in l3 children []): n93->n94 ['n93', 'n94']
l12 (in l11 children []): n83->n84 ['n83', 'n84']
l11 (in l7 children []): n85->n86 ['n82', 'n83', 'n84', 'n85', 'n86']
l19 (in l13 children []): n74->n75 ['n74', 'n75']
l28 (in l27 children []): n65->n66 ['n66', 'n65']
l27 (in l20 children []): n67->n68 ['n64', 'n65', 'n66', 'n67', 'n68']
l38 (in l29 children []): n56->n57 ['n56', 'n57']
l49 (in l39 children []): n46->n47 ['n44', 'n43', 'n47', 'n46', 'n45']
l61 (in l50 children []): n35->n36 ['n113', 'n34', 'n33', 'n32', 'n36', 'n35']
l74 (in l62 children []): n25->n26 ['n112', 'n23', 'n26', 'n24', 'n25', 'n22']
l0 (in None children []): n108->n109 ['n102', 'n103', 'n104', 'n105', 'n106', 'n107', 'n108', 'n109']
l3 (in None children []): n95->n96 ['n92', 'n93', 'n94', 'n95', 'n96']
l7 (in None children []): n87->n88 ['n81', 'n82', 'n83', 'n84', 'n85', 'n86', 'n8

In [6]:
from rwcond_out import loadstore_Obj
lds_obj = loadstore_Obj(segreader,tcfg)
lsproc = lds_obj.lsproc
ls_loop_info = lds_obj.loop_info

for k,v in ls_loop_info.items():
    print(k,v)

l2 {1056: 5.6923076923076925, 1057: 5.6923076923076925, 1058: 5.6923076923076925, 1059: 5.6923076923076925, 1060: 5.6923076923076925, 1061: 5.6923076923076925, 1062: 5.6923076923076925, 1063: 5.6923076923076925, 1064: 5.6923076923076925, 1065: 5.6923076923076925, 1066: 5.6923076923076925, 1067: 5.6923076923076925, 1068: 5.6923076923076925, 1069: 5.6923076923076925, 1070: 5.6923076923076925, 1071: 5.6923076923076925, 1072: 5.6923076923076925, 1073: 5.6923076923076925, 1074: 5.6923076923076925, 1075: 5.6923076923076925, 1076: 5.6923076923076925, 1077: 5.6923076923076925, 1078: 5.6923076923076925, 1079: 5.6923076923076925, 1080: 5.6923076923076925, 1081: 5.6923076923076925, 1082: 5.6923076923076925, 1083: 5.6923076923076925, 1084: 5.6923076923076925, 1085: 5.6923076923076925, 1086: 5.6923076923076925, 1087: 5.6923076923076925, 1088: 5.6923076923076925, 1089: 5.6923076923076925, 1090: 5.6923076923076925, 1091: 5.6923076923076925, 1092: 5.6923076923076925, 1093: 5.6923076923076925, 1094: 5.

In [7]:
# '''设置node和edge上的的value'''
# import random
# for l in tcfg.loops:
#     l.back_edge.is_backEdge = True
# for n in tcfg_nodes:
#     n.set_rw_data()

# temp_bool = True

# while temp_bool:
#     no_end = False

#     for n in tcfg_nodes:
#         n.set_rw_value()
#         if not n.no_out:
#             no_end = True
#         temp_bool = no_end

# for e in tcfg_edges:
#     if e.is_backEdge:
#         e.loop_value = random.randint(0,10)
#         e.edge_value = 2

# #for e in tcfg_edges:
# #    print(e.src.name,e.dst.name,e.edge_value)

In [8]:
# '''ls分析和读写特征'''

# from newCFG.cfg import TCfg
# from newCFG.loadstore import LSProc
# from newCFG.rw_condition import RWProc, RWType

# lsproc = LSProc(tcfg_nodes)

# lds_table = lsproc.ls_table

# for i in lds_table:
#     i.final_addr
#     i.local_offset

# #for i in lds_table:
# #    if i.target_num == 1:
#  #       pass
#         #print(i.ins.tokens,i.reg_target,i.addr_offset) 
#  #   else:
#         #print(i.ins.tokens,i.reg_target_list[0],i.reg_target_list[1],i.addr_offset,i.final_addr)

# rwproc = RWProc(lds_table)

# Global_Tolerant_value = 0
# Global_Intolerant_value = 0

# for rwu in rwproc.rw_table:
#     #print(rwu.ins.tokens,rwu.find_cycle,rwu.ins.final_addr,rwu.is_torrent)
#     if rwu.is_torrent == RWType.Global_Tolerant:
#         Global_Tolerant_value += rwu.find_cycle
#         print(rwu.ins.tokens,rwu.ins.final_addr,rwu.node.name)
# print()
# for rwu in rwproc.rw_table:
#     #print(rwu.ins.tokens,rwu.find_cycle,rwu.ins.final_addr,rwu.is_torrent)
#     if rwu.is_torrent == RWType.Global_Intolerant:
#         Global_Intolerant_value += rwu.find_cycle
#         print(rwu.ins.tokens,rwu.ins.final_addr,rwu.node.name)

# print("全局的容错路径为：",Global_Tolerant_value)
# print("全局的非容错路径为：",Global_Intolerant_value)   

In [9]:
'''cache分析'''

# 打印给cache分许需要的数据
#for i in lds_table:
#    print("指令",i.ins.tokens,"是否是SP",i.is_sp,"指令地址",i.ins.addr.val(),"访存地址",i.final_addr,"数据宽度",i.ins.ls_data_width,"是否是加入一个数组",i.ins.is_data_group)


NameError: name 'lds_table' is not defined

In [ ]:
# from newCFG.heat_analysis import loop_heat
# test=loop_heat(tcfg,lsproc,r'C:\Users\13377\Desktop\hw-memory-master (5)\hw-memory-master\benchmarks\final_benchmark\spec_benchD.asm')
# test.do_it()


In [10]:

# TODO.
from cache_analysis.cache_risk_level import CacheRisk

cache_test = CacheRisk(tcfg, lsproc, r'D:\workspace\hw-memory\benchmarks\final_benchmark\spec_benchmarkD.asm',
                       r'D:\workspace\hw-memory\cache_analysis\new_cache\input\cache_information.in')
cache_test.test()


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\workspace\\hw-memory\\benchmarks\\final_benchmark\\spec_benchmarkD.asm'

In [ ]:

# from newCFG.rw_condition import RWProc, RWType
# from newCFG.rw_condition_out import RWOut_Proc



# rwout = RWOut_Proc(tcfg_nodes,segreader,rwproc.rw_table,tcfg.loops)




